In [1]:
Threads.nthreads()

1

In [2]:
using Gridap
import Gridap: ∇
using LinearAlgebra

In [3]:
#Mallado
const global domain = (0,1,0,1)
const global partition = (20,20)
model = CartesianDiscreteModel(domain,partition)
model = simplexify(model)
const global trian = Triangulation(model)
const global degree = 1
const global quad = CellQuadrature(trian,degree)

CellQuadrature()

In [4]:
#Parámetros
const global beta = 0.95
const global nu = 0.99
const global alpha = 0.02

const global t_init = 0.
const global t_end = 2.

const global N_intervalos = 16;

const global n_iter = 5;
const global T = LinRange(t_init, t_end, N_intervalos+1);

In [5]:
#Espacio de funciones

const global V = TestFESpace(
  reffe=:Lagrangian, conformity=:H1, valuetype=Float64,
  model=model, order=1)
  
  
const global W = TrialFESpace(V)

TrialFESpace()

In [7]:
function project(q,trian,quad,order)

  print("#0")
    
  a(u,v) = u*v
  l(v) = v*q
  t_Ω = AffineFETerm(a,l,trian,quad)
  
    
  V = TestFESpace(
    reffe=:Lagrangian, valuetype=Float64, order=order,
    triangulation=trian, conformity=:L2)

  print("#1")
  U = TrialFESpace(V)
  print("#2")
  op = AffineFEOperator(U,V,t_Ω)
  print("#3")
  qh = solve(op)
  print("#4")
  qh
end

project (generic function with 1 method)

In [7]:
#writevtk(trian,"/tmp/mesh")


In [8]:
#La formula de cuadratura para la forma bilineal int(u*v), cambia el orden por ser producto de dos polinomios (cada uno de orden "degree")
const global quadL2 = CellQuadrature(trian,2*degree)

CellQuadrature()

In [ ]:
x0, y0 = 0.5, 0.5 

S0(x) = 10 -  4*exp(-200*((x[1]-x0)^2 + (x[2]-y0)^2))
I0(x) = 4*exp(-200*((x[1]-x0)^2 + (x[2]-y0)^2))
R0(x) = 0

S0_sol = project(S0, trian, quadL2, 1)
#I0_sol = project(I0, trian, quadL2, 1)
#R0_sol = project(R0, trian, quadL2, 1)

#const global U0 = [S0_sol, I0_sol, R0_sol]

In [ ]:
function EulerMEF(t_init,t_end,n_t,u0)
    S0, I0, R0 = u0
    dt = (t_end - t_init)/n_t
    t = t_init
    
    a_s(S,v) = (1 + dt*beta*I0)*S*v + dt*alpha*∇(v)*∇(S) 
    b_s(v) = v*S0
    
    a_i(I,v) = (1 - dt*beta*S0 + dt*nu)*I*v + dt * alpha*∇(v)*∇(I) 
    b_i(v) = v*I0
    
    a_r(R,v) = R*v + dt * alpha*∇(v)*∇(R)
    b_r(v) = v*(R0 + dt* nu*I0)

    
    for i=1:n_t
        t += dt
        
        t_Ω = AffineFETerm(a_s,b_s,trian,quad)
        op = AffineFEOperator(W,V,t_Ω)
        S_sol = solve(op)
        
        
        t_Ω = AffineFETerm(a_i,b_i,trian,quad)
        op = AffineFEOperator(W,V,t_Ω)
        I_sol = solve(op)
        
        
        t_Ω = AffineFETerm(a_r,b_r,trian,quad)
        op = AffineFEOperator(W,V,t_Ω)
        R_sol = solve(op)

        
        S0 = S_sol
        I0 = I_sol
        R0 = R_sol
        
    end
    return [S0, I0, R0]
end 

In [ ]:
t_n_f = 70;
const global t_n_c = 20;

@inline F(t1, t0, u0) =  EulerMEF(t0, t1, t_n_f,u0)
@inline G(t1, t0, u0) =  EulerMEF(t0, t1, t_n_c,u0)

In [ ]:
function EDP_parareal()
    U = Array{Gridap.Geometry.GenericCellField{true},3}(undef, N_intervalos+1, n_iter+1, 3);
    Fn = Array{Gridap.Geometry.GenericCellField{true},2}(undef, N_intervalos+1, 3);
    Gn = Array{Gridap.Geometry.GenericCellField{true},2}(undef, N_intervalos+1, 3);
    G0 = Array{Gridap.Geometry.GenericCellField{true},2}(undef, N_intervalos+1, 3);

    # 1.a) Inicialización (aproximción grosera)
    U[1,1,1] = 1*S0_sol
    U[1,1,2] = 1*I0_sol
    U[1,1,3] = 1*R0_sol


    for n=1:N_intervalos
        G0[n+1,1], G0[n+1, 2], G0[n+1, 3] = 1*G( T[n+1],T[n],U[n,1,:] )
        U[n+1,1,:] = G0[n+1,:]
    end
    
        
    # 1.b) Inicialización etapas parareal
    @inbounds Threads.@threads for k=1:n_iter
        U[1,k+1,1], U[1,k+1,2], U[1,k+1,3] = 1*S0_sol, 1*I0_sol, 1*R0_sol 
    end

    # 2) Bucle parareal
    for k=1:n_iter
 
        # 2.a) Aproximación fina (paralela) en cada subintervalo
        @inbounds Threads.@threads for n = 1:N_intervalos
            Fn[n,1], Fn[n,2], Fn[n,3]  = 1*F( T[n+1], T[n], U[n,k,:] )
        end
        
        # 2.b) Corrección secuencial
        @inbounds for n = 1:N_intervalos
            Gn[n+1,1], Gn[n+1, 2], Gn[n+1, 3] = 1*G( T[n+1],T[n],U[n,k+1,:] )
            U[n+1, k+1, 1] = Fn[n,1] + Gn[n+1,1] - G0[n+1,1]
            U[n+1, k+1, 2] = Fn[n,2] + Gn[n+1,2] - G0[n+1,2]
            U[n+1, k+1, 3] = Fn[n,3] + Gn[n+1,3] - G0[n+1,3]
        end
        G0 = Gn
    end
    
    return U
    
end

In [ ]:
U_SIR = EDP_parareal();

In [ ]:
using Plots

In [ ]:
suma_S = [ sum(integrate(U_SIR[i,end,1],trian,quad)) for i in 1:N_intervalos+1]
suma_I = [ sum(integrate(U_SIR[i,end,2],trian,quad)) for i in 1:N_intervalos+1]
suma_R = [ sum(integrate(U_SIR[i,end,3],trian,quad)) for i in 1:N_intervalos+1];

In [ ]:
vector_sumas = [suma_S,suma_I, suma_R ];
vector_x = 1:N_intervalos;

In [ ]:
plot(T, vector_sumas, label = ["Susceptibles" "Infectados" "Recuperados"], title="Evolución epidemia", xlabel="Tiempo", ylabel="Nº de individuos")

In [ ]:
for i=1:N_intervalos+1
    sol = U_SIR[i,end,1]
    writevtk(trian,"Susceptibles$(lpad(i,2,'0'))",cellfields=["sol" => sol])
end 

In [ ]:
for i=1:N_intervalos+1
    sol = U_SIR[i,end,2]
    writevtk(trian,"Infectados$(lpad(i,2,'0'))",cellfields=["sol" => sol])
end 

In [ ]:
for i=1:N_intervalos+1
    sol = U_SIR[i,end,3]
    writevtk(trian,"Recuperados$(lpad(i,2,'0'))",cellfields=["sol" => sol])
end 

## TIEMPOS PARAREAL-EULER

In [ ]:
function EulerMEF2(t_init,t_end,n_t,t_n_f,u0)
    U = Array{Gridap.Geometry.GenericCellField{true},2}(undef, N_intervalos+1, 3);
    S0, I0, R0 = u0
    dt = (t_end - t_init)/n_t
    t = t_init
    
    U[1,1] = 1*S0_sol
    U[1,2] = 1*I0_sol
    U[1,3] = 1*R0_sol
    
    a_s(S,v) = (1 + dt*beta*I0)*S*v + dt*alpha*∇(v)*∇(S) 
    b_s(v) = v*S0
    
    a_i(I,v) = (1 - dt*beta*S0 + dt*nu)*I*v + dt * alpha*∇(v)*∇(I) 
    b_i(v) = v*I0
    
    a_r(R,v) = R*v + dt * alpha*∇(v)*∇(R)
    b_r(v) = v*(R0 + dt* nu*I0)

    cont = 2
    
    for i=1:n_t
        t += dt
        
        t_Ω = AffineFETerm(a_s,b_s,trian,quad)
        op = AffineFEOperator(W,V,t_Ω)
        S_sol = solve(op)
        
        
        t_Ω = AffineFETerm(a_i,b_i,trian,quad)
        op = AffineFEOperator(W,V,t_Ω)
        I_sol = solve(op)
        
        
        t_Ω = AffineFETerm(a_r,b_r,trian,quad)
        op = AffineFEOperator(W,V,t_Ω)
        R_sol = solve(op)
        
        if i%t_n_f ==0
            U[cont,1] = 1*S_sol
            U[cont,2] = 1*I_sol
            U[cont,3] = 1*R_sol
            cont +=1
        end
        
        S0 = S_sol
        I0 = I_sol
        R0 = R_sol
        
    end
    return U
end 

In [ ]:
t_n_euler = N_intervalos*t_n_f

In [ ]:
tiempos = Array{Float64,2}(undef, 5,2)
for i=1:5
    t_n_f = 200 + (i-1)*100
    t_n_euler = N_intervalos*t_n_f
    t1 = @elapsed EulerMEF2(t_init,t_end,t_n_euler,t_n_f,U0)
    t2 = @elapsed EDP_parareal()
    tiempos[i,1] = t1
    tiempos[i,2] = t2
end

In [ ]:
x_tiempos = LinRange(200, 600, 5);

In [ ]:
using Plots

In [ ]:
plot(x_tiempos, tiempos, label = ["Euler" "Parareal"])

## ERROR PARAREAL FRENTE EULER

In [ ]:
U_EULER = EulerMEF2(t_init,t_end,t_n_euler,t_n_f,U0)

In [ ]:
error = U_EULER - U_SIR[:,end,:]

In [ ]:
for i=1:N_intervalos+1
    errS = error[i,1]
    writevtk(trian,"ErrorSuscep$(lpad(i,2,'0'))",cellfields=["error_S" => errS])
end 

for i=1:N_intervalos+1
    errI = error[i,2]
    writevtk(trian,"ErrorInfec$(lpad(i,2,'0'))",cellfields=["error_I" => errI])
end 

for i=1:N_intervalos+1
    errR = error[i,3]
    writevtk(trian,"ErrorRecup$(lpad(i,2,'0'))",cellfields=["error_R" => errR])
end 

In [ ]:
L2(w) = w*w

In [ ]:
error_s = error[:,1];
error_i = error[:,2];
error_r = error[:,3];

In [ ]:
error_S_L2 = [sqrt(sum( integrate(L2(i),trian,quad) )) for i in error_s];
error_I_L2 = [sqrt(sum( integrate(L2(i),trian,quad) )) for i in error_i];
error_R_L2 = [sqrt(sum( integrate(L2(i),trian,quad) )) for i in error_r];

In [ ]:
using Plots

In [ ]:
y = [error_S_L2,error_I_L2,error_R_L2]
plot(T, y)

In [ ]:
sum(integrate(U_SIR[end,end,1],trian,quad) ) + sum(integrate(U_SIR[end,end,2],trian,quad) ) + sum(integrate(U_SIR[end,end,3],trian,quad) )

In [ ]:
maximum(error_R_L2)

In [ ]:
maximum(error_S_L2)

In [ ]:
maximum(error_I_L2)